In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

In [4]:
# pulling the data from the csv file I created with 3 sensors
censors_df = pd.read_csv("censors2019_chosen3.csv")
censors_df.head()

,Date,Hour,Bourke Street Mall (North),Flinders St-Spark La,QV Market-Elizabeth St (West)
0,1/01/2019,0,918,802,637
1,1/01/2019,1,995,374,719
2,1/01/2019,2,416,98,370
3,1/01/2019,3,382,59,256
4,1/01/2019,4,165,10,197


In [5]:
# Do all 3 sensors contain counts of -1?
print(censors_df['Bourke Street Mall (North)'].min())
print(censors_df['Flinders St-Spark La'].min())
print(censors_df['QV Market-Elizabeth St (West)'].min())

-1
-1
-1


In [6]:
censors_df.count()

Date                             8760
Hour                             8760
Bourke Street Mall (North)       8760
Flinders St-Spark La             8760
QV Market-Elizabeth St (West)    8760
dtype: int64

In [7]:
censors_df.dtypes

Date                             object
Hour                              int64
Bourke Street Mall (North)        int64
Flinders St-Spark La              int64
QV Market-Elizabeth St (West)     int64
dtype: object

In [8]:
# Replacing any negative values with 0
clean_df = censors_df.mask(censors_df['Bourke Street Mall (North)'] < 0, 0)
clean_df = clean_df.mask(censors_df['Flinders St-Spark La'] < 0, 0)
clean_df = clean_df.mask(censors_df['QV Market-Elizabeth St (West)'] < 0, 0)
clean_df.head()

,Date,Hour,Bourke Street Mall (North),Flinders St-Spark La,QV Market-Elizabeth St (West)
0,1/01/2019,0,918,802,637
1,1/01/2019,1,995,374,719
2,1/01/2019,2,416,98,370
3,1/01/2019,3,382,59,256
4,1/01/2019,4,165,10,197


In [9]:
# Checking min values for each location
print(clean_df['Bourke Street Mall (North)'].max())
print(clean_df['Bourke Street Mall (North)'].min())
print(clean_df['Flinders St-Spark La'].max())
print(clean_df['Flinders St-Spark La'].min())
print(clean_df['QV Market-Elizabeth St (West)'].max())
print(clean_df['QV Market-Elizabeth St (West)'].min())

4896
0
3708
0
2957
0


In [10]:
# grouping by the date and applying aggregates
average_daily = clean_df.groupby(["Date"])
average_daily = average_daily[['Bourke Street Mall (North)','Flinders St-Spark La', 'QV Market-Elizabeth St (West)']].mean()
average_daily.head()

,Bourke Street Mall (North),Flinders St-Spark La,QV Market-Elizabeth St (West)
Date,,,
0,0.000000,0.000000,0.000000
1/01/2019,1119.375000,117.791667,363.500000
1/02/2019,1504.125000,185.708333,586.250000
1/03/2019,1458.750000,154.833333,551.000000
1/04/2019,1283.727273,138.818182,512.181818


In [11]:
# Dropping the row of zero values, not sure why that's there?
average_daily = average_daily.drop(average_daily.index[0])
average_daily

,Bourke Street Mall (North),Flinders St-Spark La,QV Market-Elizabeth St (West)
Date,,,
1/01/2019,1119.375000,117.791667,363.500000
1/02/2019,1504.125000,185.708333,586.250000
1/03/2019,1458.750000,154.833333,551.000000
1/04/2019,1283.727273,138.818182,512.181818
1/05/2019,1384.791667,180.416667,588.875000
...,...,...,...
9/08/2019,1438.166667,138.791667,577.958333
9/09/2019,1133.666667,133.625000,383.375000
9/10/2019,1304.083333,146.083333,583.583333


In [12]:
print(average_daily['Bourke Street Mall (North)'].min())
print(average_daily['Bourke Street Mall (North)'].max())
print(average_daily['Flinders St-Spark La'].min())
print(average_daily['Flinders St-Spark La'].max())
print(average_daily['QV Market-Elizabeth St (West)'].min())
print(average_daily['QV Market-Elizabeth St (West)'].max())

132.0
2235.375
17.5
781.375
99.5
1087.7916666666667


In [17]:
average_daily.to_csv("censors2019_chosen3_dailyaverage_nottransposed.csv", encoding="utf-8", index=False, header=True)

In [18]:
# Transpose the data ready to merge to the Zomato data
average_transposed = average_daily.T
average_transposed

Date,1/01/2019,1/02/2019,1/03/2019,1/04/2019,1/05/2019,1/06/2019,1/07/2019,1/08/2019,1/09/2019,1/10/2019,...,9/03/2019,9/04/2019,9/05/2019,9/06/2019,9/07/2019,9/08/2019,9/09/2019,9/10/2019,9/11/2019,9/12/2019
Bourke Street Mall (North),1119.375000,1504.125000,1458.750000,1283.727273,1384.791667,1442.583333,1296.041667,1368.375000,1104.958333,1553.260870,...,1546.083333,1355.375000,1364.666667,1340.041667,1152.916667,1438.166667,1133.666667,1304.083333,1299.125000,1303.541667
Flinders St-Spark La,117.791667,185.708333,154.833333,138.818182,180.416667,101.375000,147.750000,143.000000,77.041667,162.478261,...,134.166667,181.666667,148.958333,111.083333,184.291667,138.791667,133.625000,146.083333,105.333333,135.250000
QV Market-Elizabeth St (West),363.500000,586.250000,551.000000,512.181818,588.875000,683.333333,388.500000,632.958333,582.208333,655.913043,...,769.000000,625.916667,581.208333,734.708333,703.958333,577.958333,383.375000,583.583333,633.041667,367.500000


In [19]:
average_transposed.to_csv("censors2019_chosen3_dailyaverage.csv", encoding="utf-8", index=False, header=True)